In [14]:
using Plots
gr()

Plots.GRBackend()

In [62]:
module SP

import Base: getindex, endof

using StaticArrays

immutable SPolynomial{N, T, S}
    var::Val{S}
    coeffs::SVector{N, T}
end

(::Type{SPolynomial{N, T, S}}){N, T, S}(coeffs::NTuple{N, T}) = SPolynomial{N, T, S}(Val{S}(), coeffs)
SPolynomial{S}(v::Val{S}, coeffs) = SPolynomial(v, SVector(coeffs))

SPolynomial(var::Symbol, coeffs) = SPolynomial(Val{var}(), SVector(coeffs))

(p::SPolynomial)(x) = evaluate(p, x)

evaluate{T, S}(p::SPolynomial{0, T}, x::S) = zero(promote_type(T, S))

getindex{N, T}(p::SPolynomial{N, T}, i) = (i + 1 > N) ? zero(T) : p.coeffs[i + 1]
endof(p::SPolynomial) = degree(p)
degree(p::SPolynomial) = degree(typeof(p))
degree{N, T, S}(p::Type{SPolynomial{N, T, S}}) = N - 1

evaluate(p::SPolynomial{1}, args...) = p[0]

function evaluate{N, T, S}(p::SPolynomial{N, T}, x::S)
    R = promote_type(T, S)
    y = convert(R, p[end])
    for i = (degree(p) - 1):-1:0
        y = p[i] + x * y
    end
    y
end

# TODO: do we want this? It kind of makes sense, but it's also somewhat unexpected
# derivative{T}(p::SPolynomial{2, T}) = p[1]

@generated function derivative{N, T, S}(p::SPolynomial{N, T, S})
    tup = Expr(:tuple)
    for i in 1:degree(p)
        push!(tup.args, :($i * p[$i]))
    end
    Expr(:call, :(SPolynomial), :(Val{S}()), tup)
end

end
    

SP

In [91]:
module PF

import Base: broadcast

immutable PiecewiseFunction{Breaks, F} <: Function
    breaks::Breaks
    pieces::Vector{F}
    
    function PiecewiseFunction(breaks::Breaks, pieces::Vector{F})
        @assert issorted(breaks)
        @assert length(breaks) == length(pieces) + 1
        new(breaks, pieces)
    end
end

PiecewiseFunction{Breaks, F}(breaks::Breaks, pieces::Vector{F}) = PiecewiseFunction{Breaks, F}(breaks, pieces)

(pf::PiecewiseFunction)(t) = from_above(pf, t)
    
function from_above(pf::PiecewiseFunction, t)
    i = searchsortedlast(pf.breaks, t)
    if i <= 0 || i >= length(pf.breaks)
        error("Input value $t is out of the allowable range [$(pf.breaks[1]), $(pf.breaks[end]))")
    end
    pf.pieces[i](t - pf.breaks[i])
end 

function from_below(pf::PiecewiseFunction, t)
    i = searchsortedfirst(pf.breaks, t)
    if i <= 1 || i >= length(pf.breaks) + 1
        error("Input value $t is out of the allowable range ($(pf.breaks[1]), $(pf.breaks[end])]")
    end
    pf.pieces[i - 1](t - pf.breaks[i - 1])
end

broadcast(f, pf::PiecewiseFunction) = PiecewiseFunction(pf.breaks, f.(pf.pieces))

end

PF

In [92]:
JuMP.getvalue{N, T, S}(p::SP.SPolynomial{N, T, S}) = SP.SPolynomial(Val{S}(), getvalue.(p.coeffs))

In [93]:
function require_continuity!(model, piecewise::PF.PiecewiseFunction)
    t = piecewise.breaks
    for i = 2:(length(t) - 1)
        @constraint model PF.from_below(piecewise, t[i]) .== PF.from_above(piecewise, t[i])
    end
end

require_continuity! (generic function with 5 methods)

In [94]:
function piecewise_polynomial_variable!(model, domain, dimension::Integer, degree::Integer)
    C = @variable(model, [i=1:dimension, j=1:(degree + 1), k=1:(length(domain) - 1)], basename="C")
    polys = [SP.SPolynomial(Val{:t}(), tuple([C[:,j,k] for j in 1:(degree + 1)]...)) for k in 1:(length(domain) - 1)]
    p = PF.PiecewiseFunction(domain, polys)
end

piecewise_polynomial_variable! (generic function with 2 methods)

In [98]:
using JuMP

horizon = 20
dt = 0.1
t = 0:dt:(horizon * dt)
mass = 1
g = 9.81
dim = 1

model = Model()
position = piecewise_polynomial_variable!(model, t, dim, 2)
velocity = SP.derivative.(position)
acceleration = SP.derivative.(velocity)

require_continuity!(model, position)
require_continuity!(model, velocity)

@variable model force[1:dim, 1:horizon] >= 0

@constraint model [i=1:dim, j=1:horizon] acceleration(t[j] + dt / 2)[i] == force[i, j] - mass * g

@variable model contact[1:horizon] Bin
@constraint model [i=1:dim, j=1:horizon] force[i, j] <= 100 * contact[j]
@constraint model [i=1:dim, j=1:horizon] PF.from_below(position, t[j + 1])[i] <= 10 * (1 - contact[j])
@constraint model [i=1:dim, j=1:horizon] position(t[j])[i] >= 0

initial_position = 1
initial_velocity = 0

@constraint model position(0) .== initial_position
@constraint model velocity(0) .== initial_velocity

model
solve(model)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 120 rows, 100 columns and 315 nonzeros
Variable types: 80 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 120 rows and 100 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 
Pool objective bound 0

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap -


:Optimal

In [99]:
q = getvalue.(position)
v = getvalue.(velocity)
a = getvalue.(acceleration)
z = getvalue.(contact)
f = getvalue.(force)

1×20 Array{Float64,2}:
 0.0  0.0  0.0  0.0  45.25  17.41  …  2.21  17.41  2.21  17.41  2.21  0.0

In [100]:
plot([x -> q(x)[1], x -> v(x)[1], x -> a(x)[1]], 0:0.001:(horizon*dt - 0.001), layout=(3,1))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,3.93701 30.8865,3.93752 31.1694,3.93907 31.4523,3.94166 31.7352,3.94527 32.018,3.94992 32.3009,3.95561 32.5838,3.96232 32.8666,3.97007 33.1495,3.97885 
 33.4324,3.98867 33.7153,3.99952 33.9981,4.0114 34.281,4.02431 34.5639,4.03826 34.8467,4.05324 35.1296,4.06926 35.4125,4.08631 35.6954,4.10439 35.9782,4.1235 
 36.2611,4.14365 36.544,4.16483 36.8268,4.18705 37.1097,4.21029 37.3926,4.23457 37.6755,4.25989 37.9583,4.28623 38.2412,4.31361 38.5241,4.34203 38.8069,4.37148 
 39.0898,4.40195 39.3727,4.43347 39.6555,4.46601 39.9384,4.49959 40.2213,4.53421 40.5042,4.56985 40.787,4.60653 41.0699,4.64424 41.3528,4.68299 41.6356,4.72277 
 41.9185,4.76358 42.2014,4.80543 42.4843,4.8483 42.7671,4.89222 43.05,4.93716 43.3329,4.98314 43.6157,5.03015 43.8986,5.07819 44.1815,5.12727 44.4644,5.17738 
 44.7472,5.22853 45.0301,5.2807 45.313,5.33392 45.5958,5.38816 45.8787,5.44344 46.1616,5.49975 46.4445,5.55709 46.7273,5.61547 47.0102,5.67488 47.2931,5.73532 
 47.5759,5.7968 47.8588,5.85931 48.1417,5.92285 48.4246,5.98742 48.7074,6.05303 48.9903,6.11968 49.2732,6.18735 49.556,6.25606 49.8389,6.3258 50.1218,6.39658 
 50.4047,6.46839 50.6875,6.54123 50.9704,6.6151 51.2533,6.69001 51.5361,6.76595 51.819,6.84293 52.1019,6.92093 52.3847,6.99998 52.6676,7.08005 52.9505,7.16116 
 53.2334,7.2433 53.5162,7.32647 53.7991,7.41068 54.082,7.49592 54.3648,7.58219 54.6477,7.6695 54.9306,7.75784 55.2135,7.84721 55.4963,7.93762 55.7792,8.02906 
 56.0621,8.12153 56.3449,8.21504 56.6278,8.30958 56.9107,8.40515 57.1936,8.50175 57.4764,8.59939 57.7593,8.69807 58.0422,8.79777 58.325,8.89851 58.6079,9.00028 
 58.8908,9.10309 59.1737,9.20692 59.4565,9.3118 59.7394,9.4177 60.0223,9.52464 60.3051,9.63261 60.588,9.74161 60.8709,9.85165 61.1538,9.96272 61.4366,10.0748 
 61.7195,10.188 62.0024,10.3021 62.2852,10.4173 62.5681,10.5336 62.851,10.6508 63.1339,10.7691 63.4167,10.8885 63.6996,11.0089 63.9825,11.1303 64.2653,11.2527 
 64.5482,11.3762 64.8311,11.5007 65.1139,11.6262 65.3968,11.7528 65.6797,11.8804 65.9626,12.009 66.2454,12.1387 66.5283,12.2694 66.8112,12.4011 67.094,12.5339 
 67.3769,12.6677 67.6598,12.8025 67.9427,12.9384 68.2255,13.0753 68.5084,13.2132 68.7913,13.3522 69.0741,13.4922 69.357,13.6332 69.6399,13.7753 69.9228,13.9184 
 70.2056,14.0625 70.4885,14.2077 70.7714,14.3539 71.0542,14.5011 71.3371,14.6494 71.62,14.7987 71.9029,14.949 72.1857,15.1004 72.4686,15.2528 72.7515,15.4062 
 73.0343,15.5607 73.3172,15.7162 73.6001,15.8727 73.883,16.0303 74.1658,16.1889 74.4487,16.3485 74.7316,16.5092 75.0144,16.6709 75.2973,16.8336 75.5802,16.9974 
 75.863,17.1622 76.1459,17.328 76.4288,17.4949 76.7117,17.6628 76.9945,17.8317 77.2774,18.0017 77.5603,18.1727 77.8431,18.3447 78.126,18.5177 78.4089,18.6918 
 78.6918,18.867 78.9746,19.0431 79.2575,19.2203 79.5404,19.3986 79.8232,19.5778 80.1061,19.7581 80.389,19.9395 80.6719,20.1218 80.9547,20.3052 81.2376,20.4896 
 81.5205,20.6751 81.8033,20.8616 82.0862,21.0491 82.3691,21.2377 82.652,21.4273 82.9348,21.6179 83.2177,21.8096 83.5006,22.0023 83.7834,22.196 84.0663,22.3908 
 84.3492,22.5865 84.6321,22.7834 84.9149,22.9812 85.1978,23.1801 85.4807,23.3801 85.7635,23.581 86.0464,23.783 86.3293,23.986 86.6122,24.1901 86.895,24.3952 
 87.1779,24.6013 87.4608,24.8085 87.7436,25.0167 88.0265,25.2259 88.3094,25.4362 88.5922,25.6475 88.8751,25.8598 89.158,26.0731 89.4409,26.2875 89.7237,26.503 
 90.0066,26.7194 90.2895,26.9369 90.5723,27.1554 90.8552,27.375 91.1381,27.5956 91.421,27.8172 91.7038,28.0399 91.9867,28.2636 92.2696,28.4883 92.5524,28.714 
 92.8353,28.9408 93.1182,29.1687 93.4011,29.3975 93.6839,29.6274 93.9668,29.8583 94.2497,30.0903 94.5325,30.3233 94.8154,30.5573 95.0983,30.7923 95.381